In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import matplotlib as mpl


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

import hasasia.sim as hsim
import os


In [ ]:
SHAPE = None
NREALS = 20
NFREQS = 40
NLOUDEST = 10

# CONSTRUCT = False
# JUST_DETSTATS = False
SAVEFIG = True
SET_YLIM = SAVEFIG
# TOL=0.01
# MAXBADS=5

NVARS = 6

NPSRS = 40
NSKIES = 15
RED2WHITE = 1.0
RED_GAMMA = -3

# TARGET = 'hard_time' # EDIT AS NEEDED
# TITLE = '$\\tau_\mathrm{hard}$ (Gyr)'   # EDIT AS NEEDED

# TARGET = 'gsmf_phi0' # EDIT AS NEEDED
# TITLE = 'GSMF: $\psi_0$ '   # EDIT AS NEEDED

# TARGET = 'gsmf_mchar0_log10' # EDIT AS NEEDED
# TITLE = 'GSMF: log($M_\mathrm{char,0} / \mathrm{M}_\odot$)'   # EDIT AS NEEDED

# TARGET = 'mmb_mamp_log10' # EDIT AS NEEDED
# TITLE = 'MMB: log($\mu / \mathrm{M}_\odot$)'   # EDIT AS NEEDED

# TARGET = 'mmb_scatter_dex' # EDIT AS NEEDED
# TITLE = 'MMB: $\epsilon_\mu$ (dex)'   # EDIT AS NEEDED

TARGET = 'hard_gamma_inner' # EDIT AS NEEDED
TITLE = '$\\nu_\mathrm{inner}$ '   # EDIT AS NEEDED

In [ ]:
path = '/Users/emigardiner/GWs/holodeck/output/anatomy_09B'
load_data_from_file = path+f'/{TARGET}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz' 
load_dets_from_file = path+f'/{TARGET}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}_ds' 
load_dets_from_file = load_dets_from_file+f'_r2w{RED2WHITE:.1f}_rg{RED_GAMMA:.1f}'
load_dets_from_file = load_dets_from_file+'.npz'
# save_data_to_file = path+f'/{TARGET}_v{NVARS}vars_clbrt_pta.npz' 
print(load_data_from_file)
print(load_dets_from_file)
if os.path.exists(load_data_from_file) is False:
    err = 'load data file does not exist, you need to construct it.'
    raise Exception(err)
if os.path.exists(load_dets_from_file) is False:
    err = 'load detstats file does not exist.'
    raise Exception(err)

# print(save_data_to_file)

## Get Param Names

In [ ]:
pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=SHAPE, seed=None)
param_names = pspace.param_names

## Load Data

In [ ]:
file = np.load(load_data_from_file, allow_pickle=True)
print(file.files)
data = file['data']
params = file['params']
file.close()

file = np.load(load_dets_from_file, allow_pickle=True)
print(file.files)
dsdat = file['dsdat']

file.close()


In [ ]:
for dat in dsdat:
    print(holo.utils.stats(dat['dp_bg']))

# Plot

In [ ]:

def draw_skies_vs_bg(ax, skies_ss, dp_bg, label=None,
                     color='k', mean=True):
    xx = dp_bg # shape (R,)
    if mean:
        yy = np.mean(skies_ss, axis=-1) # shape (R,)
    else: 
        yy = np.mean(skies_ss, axis=-1) # shape (R,)
    yerr = np.std(skies_ss, axis=-1) # shape (R,)

    hh = ax.errorbar(xx, yy, yerr, color=color, label=label,
                linestyle='', capsize=3, marker='o', alpha=0.5)
    return hh

def plot_dpss_vs_dpbg(dsdat, params, target_param=TARGET, use_ev=True, title=TITLE):
    colors = cm.rainbow_r(np.linspace(0, 1, len(dsdat)))
    sslabel = 'Expected Number' if use_ev else 'Detection Probability'
    fig, ax = plot.figax(xlabel='Background Detection Probability', 
                         ylabel='$\langle$ Single Source %s} $\\rangle_\mathrm{skies}$' % sslabel,
    ) #xscale='linear', yscale='linear')
    handles = []
    for ii, ds in enumerate(dsdat):
        label = '%.2f' % params[ii][target_param]
        detss = ds['ev_ss'] if use_ev else ds['dp_ss']
        hh = draw_skies_vs_bg(ax, detss, ds['dp_bg'], color=colors[ii], label=label)
        handles.append(hh)
    ax.legend(handles=handles, loc = 'lower left', title=title,
              ncols=4, title_fontsize=12)

    fig.tight_layout()
    return fig

text = 'GSMF: $\psi_0=%.2f, m_{\phi,0}=%.2f$' % ((params[0]['gsmf_phi0']), (params[0]['gsmf_mchar0_log10']))
text = text+'\nMMB: $\mu = %.2f, \epsilon_\mu=%2f$ dex' % (params[0]['mmb_mamp_log10'], params[0]['mmb_scatter_dex'])
text = text+'\n$da/dt: \gamma_\mathrm{inner}=%.2f, \\tau_\mathrm{hard}=%.2f$' % (params[0]['hard_time'], 2.5)
print(text)




fig1 = plot_dpss_vs_dpbg(dsdat, params, use_ev=True)
fig1.axes[0].text(0.99,0.01, text, transform=fig1.axes[0].transAxes, verticalalignment='bottom', horizontalalignment='right')
fig1.tight_layout()



fig2 = plot_dpss_vs_dpbg(dsdat, params, use_ev=False)
fig2.axes[0].text(0.99,0.99, text, transform=fig1.axes[0].transAxes, verticalalignment='top', horizontalalignment='right')
fig2.tight_layout()

In [ ]:

# xlabel=TITLE
# ylabel='$\gamma_\mathrm{SS}/\gamma_\mathrm{BG}$'
# colors = cm.rainbow_r(np.linspace(0, 1, len(params)))


# fig, ax = plot.figax(xscale='linear',)
# ax.set_ylabel(ylabel, fontsize=14)
# ax.set_xlabel(xlabel, fontsize=14)

# target=TARGET
# xx = []
# yy = []
# for ii, par in enumerate(params):
#     xx.append(params[ii][target])
#     dp_bg = dsdat[ii]['dp_bg']
#     dp_ss = np.mean(dsdat[ii]['dp_ss'], axis=-1)
#     yy.append(dp_ss/dp_bg)
#     for rr in range(len(yy[0])):
#         ax.scatter(xx[ii], yy[ii][rr], color=colors[ii], alpha=0.5)
# xx = np.array(xx)
# yy = np.array(yy)
# print(yy.shape)
# # for rr, yi in enumerate(yy):
# #     ax.scatter(xx,yy[:,rr])
# plot.draw_med_conf(ax, xx, yy)
# # for rr in range(len(yy[0])):
# #     ax.scatter(xx,yy[:,rr], alpha=0.5, color=colors[])

Could do the same, but vary another model parameter for color, or show some other parameter

# Get Mass Normalization

In [ ]:
# # Get SS mass normalization
# use_snr=False
# # ssmtot = []
# # ss_dp = []
# if use_snr:
#     mt_hisnr= np.zeros((NVARS, NREALS))
#     for ii,dat in enumerate(data):
#         # ssmtot.append(dat['sspar'][0])
#         snr_ss = dsdat[ii]['snr_ss'] # (F,R,S,L)
#         # print(snr_ss.shape)
#         ssmtot = dat['sspar'][0]
#         # print(ssmtot.shape)
#         # for each realization, find the frequency with the loudest single source
#         for rr in range(len(snr_ss[0])):
#             argmax = np.argmax(snr_ss[:,rr,:,:])
#             fidx, sidx, lidx = np.unravel_index(argmax, (NFREQS, NSKIES, NLOUDEST))
#             # print(f"{fidx=}, {rr=}, {sidx=}, {lidx=}")
#             mt_hisnr[ii,rr] = ssmtot[fidx, rr, lidx]

#     mt_hisnr = np.log10(mt_hisnr/MSOL)
#     vmin = np.min(mt_hisnr)
#     vmax = np.max(mt_hisnr)
#     norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

# # print(f"{vmin=}, {vmax=}")
# # print(mt_hisnr.shape)

# Any Normalization

In [ ]:

#### use mass of max-DP single source for each realization
norm_by_maxDPmass = True
if norm_by_maxDPmass:
    normcol = np.zeros((NVARS, NREALS))
    for ii, dat in enumerate(data):
        gamma_ssi = dsdat[ii]['gamma_ssi'] # (F,R,S,L)
        ssmtot = dat['sspar'][0] # (F,R,L)
        # for each realization, find the frequency, sky, and loudest with the loudest single source
        for rr in range(NREALS):
            argmax = np.argmax(gamma_ssi[:,rr,:,:])
            ff, ss, ll = np.unravel_index(argmax, (NFREQS, NSKIES, NLOUDEST))
            normcol[ii,rr] = ssmtot[ff,rr,ll]
    normcol = np.log10(normcol/MSOL)

#### could include other colorbar options here, to set normcol values

norm = mpl.colors.Normalize(vmin=np.min(normcol), vmax=np.max(normcol))

# Normalize with individual gamma ssi

In [ ]:
cc = np.zeros((NVARS, NREALS, NSKIES))
for ii, dat in enumerate(data):
    for rr in range(NREALS):
        for ss in range(NSKIES):
            argmax = np.argmax(dsdat[ii]['gamma_ssi'][:,rr,ss,:])
            fidx, lidx =  np.unravel_index(argmax, shape=(NFREQS, NLOUDEST))
            cc[ii,rr,ss] = np.log10(dat['sspar'][0,fidx,rr,lidx]/MSOL)

norm = mpl.colors.Normalize(vmin=np.min(cc), vmax=np.max(cc))

# Plot ratio vs freq

In [ ]:
mpl.rcParams.update({'font.size':8})

xlabel=TITLE
# ylabel='$\gamma_\mathrm{SS}/\gamma_\mathrm{BG}$'
ylabel = '$EV_\mathrm{SS} / DP_\mathrm{BG}$'
clabel = '$\log M / [\mathrm{M}_\odot] $)'
cmap = cm.rainbow

fig, ax = plot.figax(xscale='linear', figsize=(4,3.5))
ax.set_ylabel(ylabel, fontsize=14)
ax.set_xlabel(xlabel, fontsize=14)
ax.set_xlim(params[0][TARGET]-0.2, params[NVARS-1][TARGET]+0.05)
if SET_YLIM: ax.set_ylim(10**-3, 10**1)
if RED2WHITE is not None and RED_GAMMA is not None:
    ax.set_title(('$S_\mathrm{red}/S_\mathrm{white}=%.2f$, $\gamma_\mathrm{red}=%.2f$, %s'
                % (RED2WHITE, RED_GAMMA, TITLE)))
else: 
    ax.set_title(f"White Noise Only, {TITLE}")

target=TARGET
xx = []
yy = []
for ii, par in enumerate(params):
    xx.append(params[ii][target])
    dp_bg = np.repeat(dsdat[ii]['dp_bg'], NSKIES).reshape(NREALS, NSKIES)
    dp_ss = dsdat[ii]['ev_ss']
    yy.append(dp_ss/dp_bg)
    # for rr in range(len(yy[0])):

x1= np.array(xx)
yy = np.array(yy)

xx = np.repeat(x1, NREALS*NSKIES).reshape(NVARS, NREALS, NSKIES)
print(f"{xx.shape=}, {yy.shape=}, {cc.shape=}")
# sax = ax.scatter(xx, yy, c=cc, cmap=cmap, norm=norm, alpha=0.2,)
# cbar = plt.colorbar(sax, ax=ax, label=clabel)


print(f"{xx.shape=}, {yy.shape=}")

col = 'tab:blue'
for pp in [50, 95]:
    med, *conf = np.percentile(yy, [50, 50-pp/2, 50+pp/2], axis=(1,2))
    ax.plot(x1, med, alpha=0.5, color=col)
    ax.fill_between(x1, *conf, color=col, alpha=0.25)
# cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), label='log ($M(\mathrm{max\ EV_\mathrm{SS}}) / [\mathrm{M}_\odot]$)',)
# fig.tight_layout()
fig.tight_layout()
figloc = '/Users/emigardiner/GWs/holodeck/output/figures/red_noise'
if SAVEFIG: 
    figname = figloc+f'/ratio_vs_{TARGET}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}_ds' 
    if RED2WHITE is not None and RED_GAMMA is not None:
        figname = figname+f'_r2w{RED2WHITE:.1f}_rg{RED_GAMMA:.1f}'
    fig.savefig(figname+'.png' , bbox_inches='tight')

Individual detprobs

In [ ]:
mpl.rcParams.update({'font.size':8})
colors = cm.rainbow(np.linspace(0,1,NVARS))

fig, ax = plot.figax(xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='Detection Probability',
                     figsize=(4,3.5))
if RED2WHITE is not None and RED_GAMMA is not None:
    ax.set_title(('$S_\mathrm{red}/S_\mathrm{white}=%.2f$, $\gamma_\mathrm{red}=%.2f$, %s'
                % (RED2WHITE, RED_GAMMA, TITLE)))
else: 
    ax.set_title(f"White Noise Only, {TITLE}")
ax.set_ylim(1e-6, 2e0)
xx = data[0]['fobs_cents']*YR
label1 = 'BG DP'
label2 = '1st Loudest DP, for each sky+strain realization'
label3 = 'Total SS DP, for each sky realization'

handles = []
for ii, dsd in enumerate(dsdat):
    y1 = dsd['dp_bg'][0] # 1
    y2 = dsd['gamma_ssi'][:,:,:,0].reshape(NFREQS, NREALS*NSKIES) # F, S
    y3 = dsd['dp_ss']
    y3 = np.repeat(y3, NFREQS).reshape(NREALS*NSKIES, NFREQS)
    y3 = np.swapaxes(y3, 0, 1)

    h1 = ax.axhline(y1)
    h2 = plot.draw_med_conf_color(ax, xx, y2, color=colors[ii])
    # for ss in range(NSKIES*NREALS):
    h3 = plot.draw_med_conf_color(ax, xx, y3, color=colors[ii])
    handles.append(
        mpl.lines.Line2D([0], [0], label=f"{params[ii][TARGET]:.2f}", color=colors[ii])
    )

leg1 = ax.legend(handles=[h1,h2,h3], labels=[label1, label2, label3], loc='upper right',
                 fontsize=7)
leg2 = ax.legend(handles=handles, title=TITLE, loc='lower right')
ax.add_artist(leg1, )
fig.tight_layout()

figloc = '/Users/emigardiner/GWs/holodeck/output/figures/red_noise'
if SAVEFIG: 
    figname = figloc+f'/DP_vs_{TARGET}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}_ds' 
    if RED2WHITE is not None and RED_GAMMA is not None:
        figname = figname+f'_r2w{RED2WHITE:.1f}_rg{RED_GAMMA:.1f}'
    print(figname)
    fig.savefig(figname+'.png' , bbox_inches='tight')

In [ ]:
mpl.rcParams.update({'font.size':10})
fig, ax = plot.figax(xlabel=plot.LABEL_GW_FREQUENCY_YR,
                     ylabel=plot.LABEL_CHARACTERISTIC_STRAIN,
                     figsize=(5,4))

# if SIGMA is not None and RED_GAMMA is not None:
#     ax.set_title(('$\sigma_\mathrm{red}+%.2e$, $\gamma_\mathrm{red}=%.2f$, %s'
#                 % (SIGMA, RED_GAMMA, TITLE)))
# else: 
#     ax.set_title(f"White Noise Only, {TITLE}")
    
xx = dat['fobs_cents']*YR
handles=[]
for ii, dat in enumerate(data):
    plot.draw_ss_and_gwb(ax, xx, dat['hc_ss'], dat['hc_bg'], color=colors[ii],
                         nsamp=2)    
    handles.append(
        mpl.lines.Line2D([0], [0], label=f"{params[ii][TARGET]:.2f}", color=colors[ii])
    )
ax.legend(handles=handles, loc='lower left', title=TITLE)

if SAVEFIG: 
    figname = figloc+f'/hc_vs_{TARGET}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}_ds' 
    if RED2WHITE is not None and RED_GAMMA is not None:
        figname = figname+f'_r2w{RED2WHITE:.1e}_rg{RED_GAMMA:.1f}'
    fig.savefig(figname+'.png' , bbox_inches='tight')

# Noise

In [ ]:
mpl.rcParams.update({'font.size':8})
colors = cm.rainbow(np.linspace(0,1,NVARS))

fig, ax = plot.figax(xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='Noise',
                     figsize=(4,3.5))
if RED2WHITE is not None and RED_GAMMA is not None:
    ax.set_title(('$S_\mathrm{red}/S_\mathrm{white}=%.2f$, $\gamma_\mathrm{red}=%.2f$, %s'
                % (RED2WHITE, RED_GAMMA, TITLE)))
else: 
    ax.set_title(f"White Noise Only, {TITLE}")

fobs_cents = data[0]['fobs_cents']
dur = 1.0/fobs_cents[0]
cad = 1.0/(2*fobs_cents[-1])
# psrs = detstats._build_pta(NPSRS, SIGMA, dur, cad)

xx = fobs_cents*YR
for ii, dat in enumerate(data):
    hc_bg = dat['hc_bg']
    hc_ss = dat['hc_ss']

    # get sigma for the 0th realization
    psrs, ramp, sigma, sigmin, sigmax, = detstats.calibrate_one_pta(hc_bg[:,0], dat['fobs_cents'], NPSRS, ret_sig=True,
                                                            red_gamma=RED_GAMMA, red2white=RED2WHITE,
                                                            debug=False)
    if psrs is not None:
        sigmas = np.ones(NPSRS)*sigma

        rr=0
        y1 = detstats._total_noise(cad, sigmas, hc_ss[:,rr:rr+1], hc_bg[:,rr:rr+1], fobs_cents,
                                red_amp=ramp, red_gamma=RED_GAMMA)[0,:,:,0]
        y2 = detstats._power_spectral_density(hc_ss[:,rr:rr+1,0], fobs_cents)
        # y3 = detstats._white_noise(cad, sigmas) # (P,)
        # y4 = detstats._red_noise(RED_AMP, RED_GAMMA, fobs_cents) # (F,)
        print(f"{y1.shape=}, {y2.shape=}")
        label1 = 'total noise'
        label2 = 'hc-rest noise'
        # label3 = 'white noise'
        # label4 = 'red noise'
        h1 = plot.draw_med_conf_color(ax, xx, y1, color=colors[ii])
        h2 = ax.plot(xx, y2, color=colors[ii], linestyle='--')
        # h3 = ax.axhline(y3[0], linestyle='--', color='k', alpha=0.5)
        # h4, = ax.plot(xx, y4, color='tab:red', linestyle='--')


        # ax.legend(handles=[h1, h2, h3, h4], 
        #         labels=[label1, label2, label3, label4])
# ax.text(1.0,0., f"{RED_AMP=}, {RED_GAMMA=}", horizontalalignment='right', verticalalignment='bottom',
#         transform=ax.transAxes)
        
handles = [mpl.lines.Line2D([0], [0], label='total noise', linestyle='-'), mpl.lines.Line2D([0], [0], label='hc_rest noise', linestyle='--')]
leg1 = ax.legend(handles=handles, loc='lower left', )

handles=[]
for ii, dat in enumerate(data):
    handles.append(
        mpl.lines.Line2D([0], [0], label=f"{params[ii][TARGET]:.2f}", color=colors[ii])
    )
leg2 = ax.legend(handles=handles, bbox_to_anchor=(0,.15), title=TITLE, loc='lower left')
ax.add_artist(leg1)
if SAVEFIG: 
    figname = figloc+f'/noise_vs_{TARGET}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}_ds' 
    if RED2WHITE is not None and RED_GAMMA is not None:
        figname = figname+f'_r2w{RED2WHITE:.1f}_rg{RED_GAMMA:.1f}'
    fig.savefig(figname+'.png' , bbox_inches='tight')